In [221]:
import numpy as np
import copy
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [222]:
dataset=StringIO("""Date,Open,High,Low,Close,Volume,Trade_count,Vwap
2015-12-01 09:00:00+00:00,118.88,118.94,118.88,118.94,1145,5,118.902052
2015-12-01 09:15:00+00:00,118.77,118.77,118.77,118.77,200,1,118.77
2015-12-01 09:30:00+00:00,118.69,118.69,118.6,118.6,900,4,118.61
2015-12-01 09:45:00+00:00,118.64,118.65,118.64,118.65,3580,5,118.648883
2015-12-01 10:00:00+00:00,118.65,118.65,118.55,118.55,1820,4,118.611538
2015-12-01 10:15:00+00:00,118.55,118.6,118.55,118.6,880,5,118.5625
2015-12-01 10:30:00+00:00,118.55,118.55,118.5,118.5,1878,5,118.513312
2015-12-01 10:45:00+00:00,118.59,118.72,118.59,118.72,2499,10,118.628431
2015-12-01 11:00:00+00:00,118.71,118.9,118.71,118.9,2842,11,118.86064
2015-12-01 11:15:00+00:00,118.87,118.87,118.87,118.87,300,2,118.87
2015-12-01 11:30:00+00:00,118.78,118.8,118.76,118.8,3914,22,118.785876
2015-12-01 11:45:00+00:00,118.8,118.99,118.77,118.9,7900,37,118.893542
2015-12-01 12:00:00+00:00,118.88,118.98,118.84,118.84,6540,34,118.922648
2015-12-01 12:15:00+00:00,118.82,118.84,118.77,118.77,5603,28,118.804962
2015-12-01 12:30:00+00:00,118.77,118.89,118.76,118.88,7612,31,118.824002
""")
df = pd.read_table(dataset, sep=",")

#ip = np.array([ [1,2,3],[6,8,9],[3,4,5],[4,7,8],[4,2,5],[5,7,4] ])
#op = np.array([[2,8,4,7,2,4]])
#op = op.reshape(6,1)
ip = np.array([ [1],  [2],  [0],  [2],  [0],  [1],  [2],  [1] ])
op = np.array([ [300],[100],[200],[100],[200],[300],[100],[300] ])
num_steps = 3
num_features = 1
#ip_shaped = np.reshape(ip, newshape=(-1, num_steps, num_features))

#X = np.array([ [1,2,3, 4, 5, 6] ])
#Y = np.array([[2,3,4,5,6,7]])
print(ip)
print("--")
#ip= np.tile(ip,(50,1))
#op = np.tile(op,(50,1))
print(len(op))
#ip = np.array([ [1],[1],[1],[1],[1],[2],[1],[2]])
#op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T

ip = np.array([ [1,1,2],[1,2,1],[1,0,2],[1,2,1],[1,0,2],[1,1,2],[1,2,1],[1,1,2]])
op = np.array([ [10,    20,    10,    20,    10,    10,    20,    10 ]]).T
#ip = np.array([ [1,1,1,1,1],[0,2,2,2,2],[1,3,3,3,3],[1,4,4,4,4],[0,5,5,5,5],[0,6,6,6,6],[1,7,7,7,7],[0,8,8,8,8]])
#op = np.array([ [1,          0,          1,          1,          0,          0,          1,          0 ]]).T
_, numFeats = ip.shape

[[1]
 [2]
 [0]
 [2]
 [0]
 [1]
 [2]
 [1]]
--
8


In [223]:
def lstm_data_transform(x_data, y_data, timeSteps=2):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    #print(x_data.shape[0])
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + timeSteps

        # if index is larger than the size of the dataset, we stop
        #print(end_ix)
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        #print(x_data[i:end_ix])
        # Get only the last element of the sequency for y
        #print(y_data[end_ix])
        seq_y = y_data[i:end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
        display(X)
        print("--")
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

def lstm_data_time(x_data, y_data, timeSteps=2):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    #print(x_data.shape[0])
    x_array = []
    y_array = []

    for i in range(x_data.shape[0]):
        my2, mx2 = list(), list()
        # compute a new (sliding window) index
        end_ix = i + timeSteps

        # if index is larger than the size of the dataset, we stop
        #print(end_ix)
        if end_ix >= x_data.shape[0]:
            break

        for j in range(i, i+timeSteps):
            my1, mx1 = list(), list()
            # Get a sequence of data for x
            seq_X = x_data[j]
            # Get only the last element of the sequency for y
            seq_y = y_data[j]
            # Append the list with sequencies
            for k in range(len(seq_X)):
                my, mx = list(), list()
                mx.append(seq_X[k])
                mx1.append(mx)
                
            mx2.append(mx1)
            my2.append(seq_y)
            
        X.append(mx2)
        y.append(my2)

        #X.append(np.array(mx1))
        #y.append(np.array(my))
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)

    return x_array, y_array
#3 3 1 want
#3 3 3 now

In [224]:
timeSteps = 3
ip = np.array([ [1,10,20],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20],
               [1,10,45],[1,10,25],[1,10,30],[1,10,25],[1,10,30],[1,10,20],[1,10,25],[1,10,20]            
              ])
# op = np.array([ [1,         1,         0,         1,         0,         1,         0,         0, 
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0,
#                  1,         1,         0,         1,         0,         1,         0,         0
#                 ]]).T
op = np.array([ [25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45, 
                 25,       30,       25,       30,       20,       25,       20,       45
                ]]).T


numFeats = 3
# ip = np.array([ [1,1,2],[1,2,1],[1,0,2],[1,2,1],[1,0,2],[1,1,2],[1,2,1],[1,1,2]])
# op = np.array([ [0,    0,    0,    0,    0,    0,    0,    0 ]]).T

# ip = np.array([ [[1,2],[1,2],[1,2],[2,3]],[[1,2],[1,2],[2,3],[1,2]],
#                 [[1,2],[1,2],[0,1],[2,3]],[[1,2],[1,2],[2,3],[1,2]],
#                 [[1,2],[1,2],[0,1],[2,3]],[[1,2],[1,2],[1,2],[2,3]],
#                 [[1,2],[1,2],[2,3],[1,2]],[[1,2],[1,2],[1,2],[2,3]]])
print(ip.shape)
ipt,opt=lstm_data_time(ip,op, timeSteps)
print(ipt.shape)
print(ipt[0].shape)
print(opt.shape)

print("====")
print(opt[0])
# batchSize = len(ipt)
# [[[1.]
#   [0.]
#   [0.]]

#  [[1.]
#   [0.]
#   [0.]]

#  [[1.]
#   [0.]
#   [0.]]

(96, 3)
(93, 3, 3, 1)
(3, 3, 1)
(93, 3, 1)
====
[[25]
 [30]
 [25]]


In [225]:
def sigmoid(x, derivative=False):
    """
    Computes the element-wise sigmoid activation function for an array x.

    Args:
     `x`: the array where the function is applied
     `derivative`: if set to True will return the derivative instead of the forward pass
    """
    x_safe = x + 1e-12
    f = 1 / (1 + np.exp(-x_safe))
    
    if derivative: # Return the derivative of the function evaluated at x
        return f * (1 - f)
    else: # Return the forward pass of the function at x
        return f
    

def update_lstmData(params, dip, dop, db, lr):
    wa, ua, ba, wi, ui, bi, wf, uf, bf, wo, uo, bo = params

    t = wa.T - (dip*lr)
    wa.T[0] = t[0].T
    t = wi.T - (dip*lr)
    wi.T[0] = t[1].T
    t = wf.T - (dip*lr)
    wf.T[0] = t[2].T
    t = wo.T - (dip*lr)
    wo.T[0] = t[3].T
    ua = ua.T - (dop*lr)[0]
    ui = ui.T - (dop*lr)[1]
    uf = uf.T - (dop*lr)[2]
    uo = uo.T - (dop*lr)[3]
    ba = ba.T - (db*lr)[0]
    bi = bi.T - (db*lr)[1]
    bf = bf.T - (db*lr)[2]
    bo = bo.T - (db*lr)[3]
    return wa, ua, ba, wi, ui, bi, wf, uf, bf, wo, uo, bo

    

In [226]:
class LSTM():
    def __init__(self, train_data, targets, batch_size=2, debug=1, test=1):
        #4 gates
        # input_activation = tanh(wa [inner] input + ua [inner] prev_output + ba)
        # input_gate  = sigmoid(wi [inner] input + ui [inner] prev_output + bi)
        # forget_gate = sigmoid(wf [inner] input + uf [inner] prev_output + bf)
        # output_gate = sigmoid(wo [inner] input + uo [inner] prev_output + bo)

        # 2 states
        # internal_state = (input_activation [Element wise] input_gate) + (forget_gate [Element wise] prev_internal_state)
        # output = tanh(internal_state) [Element wise] output_gate

        # To enable test mode. Batch size would be set as 2
        self.test = test
        
        # The number of records that would go inside the LSTM at one time. A sequence of records.
        self.batch_size = batch_size
        numFeats = 2 ###### CHANGE IT TO GET DYNAMICALLY FROM INPUT
        # Enable debug logs
        self.debug = debug
        
        # Training Data
        self.train_data = train_data
        # Target of training data
        self.targets = targets
        
        
        # input_activation
        self.wa = np.random.random((numFeats, 1))
        self.ua = np.random.random((1, 1))
        self.ba = np.random.random((1, 1))

        # input_gate
        self.wi = np.random.random((numFeats, 1))
        self.ui = np.random.random((1, 1))
        self.bi = np.random.random((1, 1))

        # forget_gate
        self.wf = np.random.random((numFeats, 1))
        self.uf = np.random.random((1, 1))
        self.bf = np.random.random((1, 1))

        # output_gate
        self.wo = np.random.random((numFeats, 1))
        self.uo = np.random.random((1, 1))
        self.bo = np.random.random((1, 1))

        # Forward Propogation Parameters
        self.prev_input_activation = 0
        self.prev_input_gate = 0
        self.prev_forget_gate = 0
        self.prev_output_gate = 0
        
        self.input_activation = 0
        self.input_gate = 0
        self.forget_gate = 0
        self.output_gate = 0
        self.internal_state = np.zeros((1, 1))
        self.output = np.zeros((1, 1))

        self.prev_input_activations = []
        self.prev_input_gates  = []
        self.prev_output_gates  = []
        self.prev_forget_gates  = []
        self.prev_internal_states  = []
        self.prev_outputs = []
        
        
        # Backward Propogation Parameters
        self.stacked_ip_weights = []
        self.stacked_op_weights = []
        
        self.der_internal_state_future = np.zeros((1, 1))
        self.delta_op_future = np.zeros((1, 1))
        
        self.input_weight_derivatives = 0
        self.output_weight_derivatives = 0
        self.bias_derivatives = 0
        #self.forget_gate_future = np.zeros((1, 1))

        
        #self.prev_internal_state = np.zeros((1, 1))
        #self.prev_output = np.zeros((1, 1))

        if self.test:
            self.batchSize = 2
            self.wa[0]=0.45
            self.wa[1]=0.25
            self.ua[0]=0.15
            self.ba[0]=0.2
            self.wi[0]=0.95
            self.wi[1]=0.8
            self.ui[0]=0.8
            self.bi[0]=0.65
            self.wf[0]=0.7
            self.wf[1]=0.45
            self.uf[0]=0.1
            self.bf[0]=0.15
            self.wo[0]=0.6
            self.wo[1]=0.4
            self.uo[0]=0.25
            self.bo[0]=0.1
        
    def update_lstmData(self, lr=.01):
        wa, ua, ba, wi, ui, bi, wf, uf, bf, wo, uo, bo = self.getLSTMparms()
        dip = self.input_weight_derivatives
        dop = self.output_weight_derivatives
        db = self.bias_derivatives
        
        t = wa.T - (dip*lr)
        wa.T[0] = t[0].T
        t = wi.T - (dip*lr)
        wi.T[0] = t[1].T
        t = wf.T - (dip*lr)
        wf.T[0] = t[2].T
        t = wo.T - (dip*lr)
        wo.T[0] = t[3].T
        ua = ua.T - (dop*lr)[0]
        ui = ui.T - (dop*lr)[1]
        uf = uf.T - (dop*lr)[2]
        uo = uo.T - (dop*lr)[3]
        ba = ba.T - (db*lr)[0]
        bi = bi.T - (db*lr)[1]
        bf = bf.T - (db*lr)[2]
        bo = bo.T - (db*lr)[3]
        self.wa=wa
        self.ua=ua
        self.ba=ba
        self.wi=wi
        self.ui=ui
        self.bi=bi
        self.wf=wf
        self.uf=uf
        self.bf=bf
        self.wo=wo
        self.uo=uo
        self.bo=bo

    def printLSTMparms(self):
        lstmData = self.getLSTMparms()
        print('wa:', lstmData[0].shape)
        print('ua:', lstmData[1].shape)
        print('ba:', lstmData[2].shape)
        print('wi:', lstmData[3].shape)
        print('ui:', lstmData[4].shape)
        print('bi:', lstmData[5].shape)
        print('wf:', lstmData[6].shape)
        print('uf:', lstmData[7].shape)
        print('bf:', lstmData[8].shape)
        print('wo:', lstmData[9].shape)
        print('uo:', lstmData[10].shape)
        print('bo:', lstmData[11].shape)


        print('wa:', lstmData[0])
        print('ua:', lstmData[1])
        print('ba:', lstmData[2])
        print('wi:', lstmData[3])
        print('ui:', lstmData[4])
        print('bi:', lstmData[5])
        print('wf:', lstmData[6])
        print('uf:', lstmData[7])
        print('bf:', lstmData[8])
        print('wo:', lstmData[9])
        print('uo:', lstmData[10])
        print('bo:', lstmData[11])

    def lstm_data_transform(self):
        """ Changes data to the format for LSTM training 
    for sliding window approach """
        # Prepare the list for the transformed data
        X, y = list(), list()
        # Loop of the entire data set
        #print(x_data.shape[0])
        for i in range(self.train_data.shape[0]):
            # compute a new (sliding window) index
            end_ix = i + self.batch_size

            # if index is larger than the size of the dataset, we stop
            #print(end_ix)
            ##TODOOOO - CHANGE BACK TO BELOW AFTER TESTING
            #if end_ix >= self.train_data.shape[0]:
            if end_ix > self.train_data.shape[0]:
                break
            # Get a sequence of data for x
            seq_X = self.train_data[i:end_ix]
            # Get only the last element of the sequency for y
            seq_y = self.targets[i:end_ix]
            # Append the list with sequencies
            X.append(seq_X)
            y.append(seq_y)
        # Make final arrays
        x_array = np.array(X)
        y_array = np.array(y)
        return x_array, y_array
        
    def getLSTMparms(self):
        return self.wa, self.ua, self.ba, self.wi, self.ui, self.bi, self.wf, self.uf, self.bf, self.wo, self.uo, self.bo

    def goForward(self, ipt):
        #4 gates
        # input_activation = tanh(wa [inner] input + ua [inner] prev_output + ba)
        # input_gate  = sigmoid(wi [inner] input + ui [inner] prev_output + bi)
        # forget_gate = sigmoid(wf [inner] input + uf [inner] prev_output + bf)
        # output_gate = sigmoid(wo [inner] input + uo [inner] prev_output + bo)

        # 2 states
        # internal_state = (input_activation [Element wise] input_gate) + (forget_gate [Element wise] prev_internal_state)
        # output = tanh(internal_state) [Element wise] output_gate
    
        wa, ua, ba, wi, ui, bi, wf, uf, bf, wo, uo, bo = self.getLSTMparms()
        
        po = self.output
        ps = self.internal_state
        if self.debug:
            print("wa : ", wa.T)
            print("ipt : ", ipt)
            print("ua : ", ua)
            print("po : ", po)
            print("ba : ", ba)
            
            print("ipt T shape", ipt.T.shape)
            print("po shape", po.shape)
            
           # print("incoming input = ",ippo.T)

        input_plus_prev_output = np.row_stack((ipt.T, po))
        ippo = input_plus_prev_output

            
        # input activation
        self.input_activation = np.tanh((np.inner(wa.T, ipt)) + (np.inner(ua, po)) + ba)
        ia = self.input_activation
        self.prev_input_activations.append(ia)

        # input gate
        self.input_gate = sigmoid((np.inner(wi.T, ipt)) + (np.inner(ui, po)) + bi)
        self.prev_input_gates.append(self.input_gate)

        # forget gate
        self.forget_gate = sigmoid((np.inner(wf.T, ipt)) + (np.inner(uf, po)) + bf)
        self.prev_forget_gates.append(self.forget_gate)
        
        # output gate
        self.output_gate = sigmoid((np.inner(wo.T, ipt)) + (np.inner(uo, po)) + bo)
        self.prev_output_gates.append(self.output_gate)

        # internal state
        self.internal_state = (np.multiply(ia, self.input_gate)) + (np.multiply(self.forget_gate, ps))
        self.prev_internal_states.append(self.internal_state)

        # output
        self.output = np.multiply(np.tanh(self.internal_state), self.output_gate)
        self.prev_outputs.append(self.output)
        
        if self.debug:
            print("input_activation = ",ia)
            print("input gate : ", self.input_gate)
            print("forget gate : ", self.forget_gate)
            print("output gate : ",self.output_gate)
            print("internal state", self.internal_state)
            print("output = ",self.output)
            print("----------------------------------")

    def stackWeights(self):
        stacked_ip_weights = np.copy(self.wa)
        stacked_ip_weights = np.column_stack((stacked_ip_weights, self.wi))
        stacked_ip_weights = np.column_stack((stacked_ip_weights, self.wf))
        stacked_ip_weights = np.column_stack((stacked_ip_weights, self.wo))
        self.stacked_ip_weights = stacked_ip_weights
        
        stacked_op_weights = np.copy(self.ua)
        stacked_op_weights = np.column_stack((stacked_op_weights, self.ui))
        stacked_op_weights = np.column_stack((stacked_op_weights, self.uf))
        stacked_op_weights = np.column_stack((stacked_op_weights, self.uo))
        self.stacked_op_weights = stacked_op_weights

    def travelBack(self, targets, inputs):

        # Unpack parameters
        wa, ua, ba, wi, ui, bi, wf, uf, bf, wo, uo, bo = self.getLSTMparms()
        tempo = np.zeros((1, 1))

        for t in reversed(range(len(self.prev_outputs))):

            output = self.prev_outputs[t]
            target = targets[t]
            
            next_forget_gate = np.zeros((1, 1)) if (t==len(self.prev_outputs)-1) else self.prev_forget_gates[t+1]
            
            if self.debug:
                print("previous outputs = ", str(self.prev_outputs))
                print("target = ",str(target))
                print("output = ", str(output))
            
            # Track loss
            loss = (np.power((target - output),2))/2
            print("loss = ", str(loss))

            # derivative of loss with respect to output
            der_loss_wrt_output = output - target
            print("der_loss_wrt_output = ", der_loss_wrt_output)

            # derivative of output
            der_output = der_loss_wrt_output + self.delta_op_future
            print("der_output = ", der_output)


#             print("\nDEBUG")
#             print("der output : ", str(der_output))
#             print("pog : ", pog)
#             print("ps : ", ps)
#             print("next state : ", next_state)
#             print("next forget : ", next_forget)
#             #print("output : ", output)
#             print("DEBUG\n")
            # derivative of internal state
            pog = self.prev_output_gates[t]
            ps = self.prev_internal_states[t]
            dfis = der_output * pog * (1 - (np.tanh(ps))**2 ) + (self.der_internal_state_future * next_forget_gate)
            self.der_internal_state_future = dfis
            print("der internal state = ", dfis)

            pig = self.prev_input_gates[t]
            pia = self.prev_input_activations[t]
            der_input_activation = dfis * pig * (1 - pia**2)
            print("der_input_activation = ", der_input_activation)
            stacked_ders = np.copy(der_input_activation)

            der_inputg = dfis * pia * pig * (1 - pig)
            stacked_ders = np.row_stack((stacked_ders, der_inputg))
            print("der_input = ", der_inputg)

            pps = tempo if t==0 else self.prev_internal_states[t-1] 
            pfg = self.prev_forget_gates[t]
            der_forgetg = dfis * pps * pfg * (1 - pfg)
            stacked_ders = np.row_stack((stacked_ders, der_forgetg))
            print("der_forget = ", der_forgetg)   
            
            print("\nDEBUG")
            print("pps : ", pps, t-1)
            print("pfg : ", pfg)
            print("dfis : ", str(dfis))
            #print("pog : ", pog)
            
            #print("next state : ", next_state)
            #print("next forget : ", next_forget)
            #print("output : ", output)
            print("DEBUG\n")

            der_outputg = der_output * np.tanh(ps) * pog * (1 - pog)
            stacked_ders = np.row_stack((stacked_ders, der_outputg))
            print("der_output = ", der_outputg)

            self.stackWeights()


#         print("\nstacked derivatives shape = ",stacked_ders.shape)
#         print("stacked derivatives = ",stacked_ders)
#         print("\nstacked input weights shape = ",stacked_ip_weights.shape)
#         print("stacked input weights = ",stacked_ip_weights)
#         print("\nstacked output weights shape = ",stacked_op_weights.shape)
#         print("stacked output weights = ",stacked_op_weights)

            der_input_state = np.dot(self.stacked_ip_weights, stacked_ders)
            print("der_input_state = ", der_input_state)

            der_output_state = np.dot(self.stacked_op_weights, stacked_ders)
            print("der_output_state = ", der_output_state)
            self.delta_op_future = der_output_state

            der_input_weight = np.dot(stacked_ders, inputs[t])
            self.input_weight_derivatives += der_input_weight
            print("der_input_weight : ", der_input_weight)

            po = tempo if t==0 else self.prev_outputs[t-1] 
            der_op_weight = np.dot(stacked_ders, po)
            self.output_weight_derivatives += der_op_weight
            print("der_op_weight : ", der_op_weight)

            self.bias_derivatives += stacked_ders
            print("------------------------------------------")
    
    def train(self):
        ip_batches, op_batches = self.lstm_data_transform()
        print(op_batches)
        for ip in ip_batches:
            print("1")
            self.goForward(ip)
        print("IP DONE")
        #for op in reversed(op_batches):
        self.travelBack(op_batches, ip_batches)
        print("OP DONE")
        print("OLD WEIGHTS")
        self.printLSTMparms()
        self.update_lstmData(lr=.01)
        print("NEW WEIGHTS")
        self.printLSTMparms()
        

In [227]:
ip1=np.array([[1,2]])
ip2=np.array([[0.5,3]])
ip = np.array([[1,2],[0.5,3]])
op1=np.array([[0.5]])
op2=np.array([[1.25]])
op= np.array([ [0.5],[1.25] ])

lstm = LSTM(train_data=ip, targets=op, batch_size=1, debug=1, test=1)
lstm.train()

[[[0.5 ]]

 [[1.25]]]
1
wa :  [[0.45 0.25]]
ipt :  [[1. 2.]]
ua :  [[0.15]]
po :  [[0.]]
ba :  [[0.2]]
ipt T shape (2, 1)
po shape (1, 1)
input_activation =  [[0.81775408]]
input gate :  [[0.96083428]]
forget gate :  [[0.8519528]]
output gate :  [[0.81757448]]
internal state [[0.78572615]]
output =  [[0.5363134]]
----------------------------------
1
wa :  [[0.45 0.25]]
ipt :  [[0.5 3. ]]
ua :  [[0.15]]
po :  [[0.5363134]]
ba :  [[0.2]]
ipt T shape (2, 1)
po shape (1, 1)
input_activation =  [[0.84980402]]
input gate :  [[0.98118397]]
forget gate :  [[0.87030197]]
output gate :  [[0.84993334]]
internal state [[1.5176331]]
output =  [[0.77198111]]
----------------------------------
IP DONE
previous outputs =  [array([[0.5363134]]), array([[0.77198111]])]
target =  [[1.25]]
output =  [[0.77198111]]
loss =  [[0.11425103]]
der_loss_wrt_output =  [[-0.47801889]]
der_output =  [[-0.47801889]]
der internal state =  [[-0.07110771]]
der_input_activation =  [[-0.01938435]]
der_input =  [[-0.001115